In [1]:
# The essentials
import pandas as pd
import numpy as np

# Plotting
%matplotlib inline
import matplotlib.pyplot as plt

# Progress bars
from tqdm import tqdm

# Gradient Boosting
from catboost import CatBoostRegressor

from collections import defaultdict

In [2]:
train_df = pd.read_csv('./data/train_values.csv', index_col=0, parse_dates=['timestamp'])
test_df = pd.read_csv('./data/test_values.csv', index_col=0, parse_dates=['timestamp'])
label_df = pd.read_csv('./data/train_labels.csv', index_col='process_id')



/home/kiani/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [9]:
ts_cols = [
    'process_id',
    'supply_flow',
    'supply_pressure',
    'return_temperature',
    'return_conductivity',
    'return_turbidity',
    'return_flow',
    'tank_level_pre_rinse',
    'tank_level_caustic',
    'tank_level_acid',
    'tank_level_clean_water',
    'tank_temperature_pre_rinse',
    'tank_temperature_caustic',
    'tank_temperature_acid',
    'tank_concentration_caustic',
    'tank_concentration_acid',
    'timestamp'
]

unique_training_process_ids = train_df['process_id'].unique()[0:250]
unique_test_process_ids = test_df['process_id'].unique()[0:250]
train_df_subset = train_df[train_df['process_id'].isin(unique_training_process_ids)]
test_df_subset = test_df[test_df['process_id'].isin(unique_test_process_ids)]
label_df_subset = label_df.loc[unique_training_process_ids]

In [13]:
params = EfficientFCParameters() 
subset_func = ['cwt_coefficients', 'fft_coefficient', 'fft_aggregated']
filtered_params = {}
for func in subset_func: 
    filtered_params[func] = params[func]


dict_keys(['variance_larger_than_standard_deviation', 'has_duplicate_max', 'has_duplicate_min', 'has_duplicate', 'sum_values', 'abs_energy', 'mean_abs_change', 'mean_change', 'mean_second_derivative_central', 'median', 'mean', 'length', 'standard_deviation', 'variance', 'skewness', 'kurtosis', 'absolute_sum_of_changes', 'longest_strike_below_mean', 'longest_strike_above_mean', 'count_above_mean', 'count_below_mean', 'last_location_of_maximum', 'first_location_of_maximum', 'last_location_of_minimum', 'first_location_of_minimum', 'percentage_of_reoccurring_datapoints_to_all_datapoints', 'percentage_of_reoccurring_values_to_all_values', 'sum_of_reoccurring_values', 'sum_of_reoccurring_data_points', 'ratio_value_number_to_time_series_length', 'maximum', 'minimum', 'time_reversal_asymmetry_statistic', 'c3', 'cid_ce', 'symmetry_looking', 'large_standard_deviation', 'quantile', 'autocorrelation', 'agg_autocorrelation', 'partial_autocorrelation', 'number_cwt_peaks', 'number_peaks', 'binned_ent

In [15]:
from tsfresh import extract_features
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.utilities.dataframe_functions import impute

extracted_features = extract_features(train_df_subset[ts_cols], impute_function=impute, column_id="process_id", column_sort="timestamp", default_fc_parameters=filtered_params)


Feature Extraction:   0%|          | 0/10 [00:00<?, ?it/s]

Feature Extraction:  10%|█         | 1/10 [00:02<00:18,  2.09s/it]

Feature Extraction:  20%|██        | 2/10 [00:02<00:12,  1.52s/it]

Feature Extraction:  30%|███       | 3/10 [00:03<00:10,  1.52s/it]

Feature Extraction:  40%|████      | 4/10 [00:04<00:07,  1.19s/it]

...

Feature Extraction:  60%|██████    | 6/10 [00:06<00:04,  1.17s/it]

Feature Extraction:  70%|███████   | 7/10 [00:08<00:03,  1.21s/it]

Feature Extraction:  80%|████████  | 8/10 [00:08<00:01,  1.03it/s]

Feature Extraction:  90%|█████████ | 9/10 [00:09<00:01,  1.09s/it]

Feature Extraction: 100%|██████████| 10/10 [00:10<00:00,  1.17it/s]

In [20]:
extracted_features.shape

(250, 6960)

In [8]:
# process_comb = 15
# test_features_index = pd.read_csv('./data/test_features_{}.csv'.format(process_comb), index_col=['process_id']).index
# train_features_index = pd.read_csv('./data/train_features_adv_{}.csv'.format(process_comb), index_col=['process_id']).index
# val_features_index = pd.read_csv('./data/val_features_adv_{}.csv'.format(process_comb), index_col=['process_id']).index

In [9]:
# train_df_no_final = train_df[train_df['phase'] != 'final_rinse']
# 
# X_train = create_feature_matrix(train_df_no_final[train_df_no_final['process_id'].isin(train_features_index)])
# X_val = create_feature_matrix(train_df_no_final[train_df_no_final['process_id'].isin(val_features_index)])
# X_test = create_feature_matrix(test_df[test_df['process_id'].isin(test_features_index)])

  0%|          | 0/3354 [00:00<?, ?it/s]

  0%|          | 7/3354 [00:00<00:52, 64.16it/s]

  0%|          | 14/3354 [00:00<00:51, 64.30it/s]

  1%|          | 21/3354 [00:00<00:51, 65.06it/s]

  1%|          | 28/3354 [00:00<00:51, 64.57it/s]

...

 94%|█████████▍| 630/671 [00:06<00:00, 125.50it/s]

 96%|█████████▌| 643/671 [00:06<00:00, 125.75it/s]

 98%|█████████▊| 657/671 [00:06<00:00, 127.09it/s]

100%|█████████▉| 670/671 [00:06<00:00, 125.28it/s]

100%|██████████| 671/671 [00:06<00:00, 101.95it/s]

In [19]:
# # variables we'll use to create our time series features

# 
# # variables for binary time series features
# bin_cols = [
#     'supply_pump',
#     'supply_pre_rinse',
#     'supply_caustic',
#     'return_caustic',
#     'supply_acid',
#     'return_acid',
#     'supply_clean_water',
#     'return_recovery_water',
#     'return_drain',
#     'object_low_level'
# ]
# 
# # phases, ordered from earliest to latest
# phases = ['pre_rinse', 'caustic', 'intermediate_rinse', 'acid']
# 
# def encode_categorical(df):
#     # Currently just copy-pasted from http://drivendata.co/blog/rinse-over-run-benchmark/
#     
#     # select process_id and pipeline
#     meta = df[['process_id', 'pipeline']].drop_duplicates().set_index('process_id') 
#     
#     # convert categorical pipeline data to dummy variables
#     meta = pd.get_dummies(meta)
#     
#     # pipeline L12 not in test data (so useless feature)
#     if 'pipeline_L12' in meta:
#         meta = meta.drop('pipeline_L12', axis=1)
#     
#     # calculate number of phases for each process_object
#     meta['num_phases'] = df.groupby('process_id')['phase'].apply(lambda x: x.nunique())
#     
#     return meta
# 
# def percentile_25(x):
#   return np.percentile(x, 0.25)
# 
# def percentile_75(x):
#   return np.percentile(x, 0.75)
#   
# def encode_real_timeseries(df):   
#     ts_df = df[ts_cols].set_index('process_id')
#     
#     # create features: count, min, max, mean, standard deviation
#     ts_features = ts_df.groupby('process_id').agg(['min', 'max', 'mean', 'std', 
#                                                    'count', 'median', 'sum', 
#                                                    'mad', percentile_25,
#                                                    percentile_75])
#     
#     # Now we will get the mean + variance value of the last K measurements for each phase
#     all_vals_per_phase = []
#     K = 5
#     col_names = ['process_id'] 
#     for phase in phases:
#         for col in ts_cols:
#             col_names.extend(['mean_{}_{}_{}'.format(col, K, phase), 
#                               'std_{}_{}_{}'.format(col, K, phase)])
#     for process in tqdm(ts_features.index, total=len(ts_features)):
#         vals_per_phase = [process]
#         process_filtered_df = df[df['process_id'] == process]
#         for phase in phases:
#             filtered_df = process_filtered_df[process_filtered_df['phase'] == phase].tail(K)
#             for col in ts_cols:
#                 vals_per_phase.extend([filtered_df[col].mean(), filtered_df[col].std()])
#                 
#         all_vals_per_phase.append(vals_per_phase)
#     values_df = pd.DataFrame(all_vals_per_phase, columns=col_names)
#     values_df = values_df.set_index('process_id')
#     
#     ts_features = ts_features.merge(values_df, left_index=True, right_index=True)
#     
#     return ts_features
# 
# def encode_binary_timeseries(df):
#     ts_df = df[ts_cols].set_index('process_id')
#             
#     # create features: count, min, max, mean, standard deviation
#     ts_features = ts_df.groupby('process_id').agg(['mean', 'std', 'count', 'sum', 'mad'])
#     
#     # TODO: Count fraction of True in each phase
#     feature_vectors = []
#     col_names = ['process_id'] 
#     for phase in phases:
#         for col in bin_cols:
#             col_names.append('fraction_{}_{}'.format(col, phase))
#             
#     # Get fraction of True values for each binary timeseries
#     for process in tqdm(set(df['process_id']), total=len(set(df['process_id']))):
#         vector = [process]
#         process_filtered_df = df[df['process_id'] == process]
#         for phase in phases:
#             filtered_df = process_filtered_df[process_filtered_df['phase'] == phase]
#             for col in bin_cols:
#                 if len(filtered_df):
#                     vector.append(sum(filtered_df[col]) / len(filtered_df))
#                 else:
#                     vector.append(np.NaN)
#                 
#         feature_vectors.append(vector)
#                 
#     feature_df = pd.DataFrame(feature_vectors, columns=col_names)
#     feature_df = feature_df.set_index('process_id')
#     
#     feature_df = feature_df.merge(ts_features, left_index=True, right_index=True)
#     
#     return feature_df
# 
# def create_feature_matrix(df):
#     metadata = encode_categorical(df)
#     time_series = encode_real_timeseries(df)
#     binary_features = encode_binary_timeseries(df)
#     
#     # join metadata and time series features into a single dataframe
#     feature_matrix = metadata
#     feature_matrix = feature_matrix.merge(time_series, left_index=True, right_index=True)
#     feature_matrix = feature_matrix.merge(binary_features, left_index=True, right_index=True)
#     
#     return feature_matrix